In [172]:
#combining all datasets using map function

import glob
import pandas as pd 

#map renewables

df_renew = pd.concat(map(pd.read_csv, glob.glob('C:/Users/Rachel/OneDrive/Documents/Springboard/CAPSTONE 2/energy/raw_data_renewable/*.csv')))
df_renew.rename(columns={'Country or Area':'Geo'}, inplace=True)

#map nonrenewables

df_nonrenew = pd.concat(map(pd.read_csv, glob.glob('C:/Users/Rachel/OneDrive/Documents/Springboard/CAPSTONE 2/energy/raw_data_nonrenewable/*.csv')))
df_nonrenew.rename(columns={'Country or Area':'Geo'}, inplace=True)

df = pd.concat([df_renew, df_nonrenew])

#split commodity and transaction into two columns
#defining column to split and which delimiter
ct = df['Commodity - Transaction'].str.split('-')

#assigning new columns
df['Commodity'] = ct.str.get(0)
df['Transaction'] = ct.str.get(1)

#lowercase and remove trailing and leading spaces
df['Commodity'] = df['Commodity'].str.lower()
df['Commodity'] = df['Commodity'].str.strip()

df['Transaction'] = df['Transaction'].str.lower()
df['Transaction'] = df['Transaction'].str.strip()

df = df[df['Transaction'] == 'production']


In [173]:
import pandas as pd 
unit_conv = pd.read_csv('C:/Users/Rachel/OneDrive/Documents/Springboard/CAPSTONE 2/energy/renewable_energy_units_conv_2.csv')

df_merge = pd.merge(df,  
                     unit_conv,  
                     on ='Commodity', 
                     how ='left') 

df_merge['TJ'] = df_merge['Quantity'] * df_merge['Conv_mt_to_tj']
df_merge.sample(3)

,Geo,Commodity - Transaction,Year,Unit,Quantity,Quantity Footnotes,Commodity,Transaction,Description,Units,Conv_mt_to_tj,r_type,TJ
5749,Spain,Black Liquor - Production,1996.0,Terajoules,26253.0,NaN,black liquor,production,NaN,Terajoules,1.00,renewable,26253.00
18509,New Zealand,Geothermal Electricity-production,2002.0,"Kilowatt-hours, million",2814.0,NaN,geothermal electricity,production,NaN,"Kilowatt-hours, million",3.57,renewable,10045.98
30688,Sweden,Solar Electricity-Production,2017.0,"Kilowatt-hours, million",230.0,NaN,solar electricity,production,NaN,"Kilowatt-hours, million",3.57,renewable,821.10


In [174]:

#sum all energy types, df_merge is the combination of unit conversion table and raw data to sum total TJ
#filter by year
lt_2018 = df_merge[df_merge['Year'] == 2018]
#greate geo group
tot_grp_country = lt_2018.groupby(['Geo'])
#sum TJ by geo group
df_tot_country = tot_grp_country['TJ'].agg(['sum'])

#sum all renewables
lt_2018_r = df_merge[(df_merge['Year'] == 2018) & (df_merge['r_type'] == 'renewable')]
tot_r_grp_country = lt_2018_r.groupby(['Geo'])
df_tot_country_r = tot_r_grp_country['TJ'].agg(['sum'])

#sum all nonrenewables
lt_2018_n = df_merge[(df_merge['Year'] == 2018) & (df_merge['r_type'] == 'nonrenewable')]
tot_n_grp_country = lt_2018_n.groupby(['Geo'])
df_tot_country_n = tot_n_grp_country['TJ'].agg(['sum'])

In [175]:
#merge total, renewable, and nonrenewable dataframes

countries_r = pd.merge(df_tot_country,  
                     df_tot_country_r,  
                     on ='Geo', 
                     how ='left') 

countries_r_n = pd.merge(countries_r,
                    df_tot_country_n,
                    on = 'Geo', 
                    how = 'left')

countries_r_n.rename(columns={'sum_x':'total', 'sum_y': 'renew', 'sum' : 'nonrenew'}, inplace=True)
countries_r_n['percent_renewable'] = (countries_r_n['renew'] / (countries_r_n['renew'] + countries_r_n['nonrenew'])) * 100
countries_r_n.sort_values(by='percent_renewable', ascending=False, inplace=True)
#allsources_r_n['diff'] = allsources_r_n['renew'] + allsources_r_n['nonrenew']#

countries_r_n.head(10)

,total,renew,nonrenew,percent_renewable
Geo,,,,
Mauritania,2.736788e+04,2.736788e+04,0.0000,100.000000
Guinea,1.297902e+05,1.297870e+05,3.2400,99.997504
Ethiopia,1.530337e+06,1.530257e+06,79.6000,99.994799
Nepal,4.470420e+05,4.460470e+05,995.0000,99.777426
New Caledonia,1.601628e+03,1.596148e+03,5.4800,99.657848
Slovenia,1.356360e+05,1.350212e+05,614.8000,99.546728
Lao People's Dem. Rep.,5.890988e+05,5.854094e+05,3689.4000,99.373721
Rwanda,1.052503e+05,1.045564e+05,693.9000,99.340714
Malawi,8.704760e+04,8.576397e+04,1283.6274,98.525373


In [187]:
#to get the breakdown of types for the top 10 'natural' group

#sum all energy types, df_merge is the combination of unit conversion table and raw data to sum total TJ
#filter by year
for geo in df_merge['Geo'].unique() : 
    country_types = df_merge[(df_merge['Year'] == 2018) & (df_merge['Geo'] == geo) & (df_merge['r_type'] == 'renewable')]
    #greate geo group
    country_type_grp = country_types.groupby(['Commodity'])
    #sum TJ by geo group
    country_types = country_type_grp['TJ'].agg(['sum'])
    country_types.sort_values(by='sum', ascending=False, inplace=True)
    country_types.head(20)


    #comment out unless you want to export 200+ csv files:
    #country_types.to_csv('C:/Users/Rachel/OneDrive/Documents/Springboard/CAPSTONE 2/energy/renewable_types_by_country/' + geo + '.csv')

In [177]:
#export results to csv for tableau viz, keep commented out

#countries_r_n.to_csv('countries_percent_renewable_2013.csv')

In [178]:


from matplotlib import pyplot as plt 

for geo in df_merge['Geo'].unique() :
    df_r_type_country = df_merge[df_merge['Geo'] == geo]
    r_grp_country = df_r_type_country.groupby(['r_type'])
    sum_r_type = r_grp_country['TJ'].agg(['sum'])
    sum_r_type['percent'] = (sum_r_type['sum'] / sum_r_type['sum'].sum()) * 100
    sum_r_type.head() 
    
    
    
    pie = sum_r_type['sum']
    labels = ['nonrenewable', 'renewable']

    #plt.style.use("fivethirtyeight")

    plt.pie(pie, labels=labels, wedgeprops={'edgecolor':'black'})
    plt.title(str(geo))
    #plt.tight_layout()

    #plt.savefig('worldwide_allsources_2018_pie')
    #plt.show()

ValueError: 'label' must be of length 'x'